# Problem 1

## Task 1

Create an utility-maximizing agent that decides on the mode (CAR vs TRAIN vs SM) to take
given a set of alternative (mode) specific attributes and its own (agent) internal characteristics.
The agent’s action follows the multinomial logit choice model below: 

$$
𝑉_{car} = −0.608 − 0.0111 × 𝐶𝐴𝑅_𝑇𝑇 − 0.00936 × 𝐶𝐴𝑅_𝐶𝑂 − 1.88 × 𝑆𝐸𝑁𝐼𝑂𝑅 \\
𝑉_{train} = −0.0111 × 𝑇𝑅𝐴𝐼𝑁_𝑇𝑇 − 0.0268 × 𝑇𝑅𝐴𝐼𝑁_𝐶𝑂 − 0.00586 × 𝑇𝑅𝐴𝐼𝑁_𝐻𝐸 + 0.557 × 𝐺𝐴 \\
𝑉_{SM} = −0.135 − 0.0111 × 𝑆𝑀_𝑇𝑇 − 0.0104 × 𝑆𝑀 𝐶𝑂𝑆𝑇 − 0.00586 × 𝑆𝑀_𝐻𝐸 − 1.88 × 𝑆𝐸𝑁𝐼𝑂𝑅 \\
+ 0.557 × 𝐺𝐴
$$

where 𝐶𝐴𝑅_𝑇𝑇 is the car travel time, 𝐶𝐴𝑅_𝐶𝑂 is the car cost, 𝑇𝑅𝐴𝐼𝑁_𝑇𝑇 is the train travel time, 𝑇𝑅𝐴𝐼𝑁_𝐶𝑂
is the train cost (considering the ownership of Swiss annual season ticket, GA), 𝑇𝑅𝐴𝐼𝑁_𝐻𝐸 is train head-
way (in minutes), 𝑆𝑀_𝑇𝑇 is the Swissmetro travel time, 𝑆𝑀_𝐶𝑂 is the Swiss- metro cost (considering the
ownership of 𝐺𝐴), 𝑆𝑀_𝐻𝐸 is the Swissmetro headway, 𝑆𝐸𝑁𝐼𝑂𝑅 is a dummy variable for senior people
(age above 651 ) and 𝐺𝐴 a dummy variable that captures the effect of the Swiss annual season ticket for
train.

In [125]:
from enum import Enum
import numpy as np
import pandas as pd
import math

rng = np.random.default_rng(1)
# First, define the enum and class Agent that will take in the input and deterine which choice to make
# In the data, the choice is defined in this manner so we will just follow it to make life good
choices = np.array([1, 2, 3])

    
class Agent:
    def __init__(self, car_tt = 0, car_co = 0, train_tt = 0, train_co = 0, train_he = 0, sm_tt = 0, sm_co = 0, sm_he = 0, senior = 0, ga = 0, cost_mul = 1 ):
        self.car_tt = car_tt
        self.car_co = car_co
        self.train_tt = train_tt
        self.train_co = train_co
        self.train_he = train_he
        self.sm_tt = sm_tt
        self.sm_co = sm_co
        self.sm_he = sm_he
        self.senior = senior
        self.ga = ga
        self.cost_mul = cost_mul
    
    def _car_utility(self):
        return -0.608 - 0.0111 * self.car_tt - 0.00936 * self.car_co - 1.88 * self.senior
    
    def _train_utility(self):
        return -0.0111 * self.train_tt - 0.0268 * self.train_co - 0.00586 * self.train_he + 0.557 * self.ga
    
    def _SM_utility(self):
        return -0.135 - 0.0111 * self.sm_tt - 0.0104 * self.sm_co * self.cost_mul - 0.00586 * self.sm_he - 1.88 * self.senior + 0.557 * self.ga
    
    def return_choice(self):
        #In this function, we can either use the logit functions to convert it into a continuous probability curve or just see which utility is higher
        #At high mu's the answer is basically the same
        mu = 10.0
        utility_list = [self._train_utility(), self._SM_utility(), self._car_utility()]
        utility_list = [math.exp(mu * i) for i in utility_list]
        Sum = sum(utility_list)
        utility_list = [i/Sum for i in utility_list]
        choice = rng.choice(choices, p=np.array(utility_list))
        return choice
    
    

## Task 2

A. Compute the estimated market shares by both sample enumeration and simulation for the
entire sample provided in the data file (N=10728).

In [89]:
# Firstly, we have to read the dat file. For each of row of data, we extract the relevant variable and input the values into the agent class
# We would then have a list of choice based on our prediction based on our model.

#Read the .dat file as csv, change the directory according to your needs
swiss_data = pd.read_csv(
    r'E:\Downloads\Workspace\42188 Agent modelling\CS1- Agent Behaviour Simulation attached files 09 September, 2022 1047 AM\swissmetro.dat',  delimiter="\t")

In [90]:
#Check the type for AGE variable
print(swiss_data["AGE"].dtypes)

# In the data, most of the heading are just the variable but in all caps, however for senior,
# we would have to create a new column based on the AGE column in the data where anyone with AGE value 6 has SENIOR 1 and the rest is 0
# We would also to clean the senior column as there is a not know variable in which we will drop those rows

# Function to make a new senior column
def make_senior_col(row):
    if row['AGE'] == 5:
        return 1
    else:
        return 0

#Number of rows before cleaning data
print(swiss_data.shape[0])  
swiss_data.drop(swiss_data[swiss_data['AGE'] == 6].index, inplace = True)
swiss_data.drop(swiss_data[swiss_data['CHOICE'] == 0].index, inplace = True)

#Number of rows after cleaning data and you do see some rows are removed so its good we accounted for these people with weird ages
print(swiss_data.shape[0]) 

#Make new Senior column
swiss_data['SENIOR'] = swiss_data.apply(lambda row: make_senior_col(row), axis=1)

#Anyone with a 5 for AGE is 1 for SENIOR
print(swiss_data[["AGE", "SENIOR"]])

int64
10728
10710
       AGE  SENIOR
0        3       0
1        3       0
2        3       0
3        3       0
4        3       0
...    ...     ...
10723    5       1
10724    5       1
10725    5       1
10726    5       1
10727    5       1

[10710 rows x 2 columns]


In [118]:
#Now apply the agent choice into the dataframe
# Function to make a new choice column
def make_logit_choice(row, multiplier = 1):
    agent = Agent(row["CAR_TT"], row["CAR_CO"], row["TRAIN_TT"], row["TRAIN_CO"], row["TRAIN_HE"], row["SM_TT"], row["SM_CO"], row["SM_HE"], row["SENIOR"], row["GA"], multiplier)
    return agent.return_choice()

#Make new logit choice column
swiss_data['LOGIT_CHOICE'] = swiss_data.apply(lambda row: make_logit_choice(row), axis=1)

In [92]:
#Perform sample enumeration
def output(microdata):
    idx = rng.choice(np.arange(microdata.shape[0]), size=int(microdata.shape[0] * 0.4))
    #Add the row from the original data according to the index
    sample = microdata.iloc[idx]
    return sample

sample_enumeration = output(swiss_data)

B. Comment on both their values \
C. Compare against the observed market shares (computed using the CHOICE variable in the
data set).

In [93]:
# Compare share
print(f"Logit Choice: \n{swiss_data['LOGIT_CHOICE'].value_counts(normalize=True).sort_index()}")
print(f"Sample Enumeration: \n{sample_enumeration['CHOICE'].value_counts(normalize=True).sort_index()}")
print(f"Observed Market Shares: \n{swiss_data['CHOICE'].value_counts(normalize=True).sort_index()}")

Logit Choice: 
1    0.022876
2    0.589356
3    0.387768
Name: LOGIT_CHOICE, dtype: float64
Sample Enumeration: 
1    0.133287
2    0.592204
3    0.274510
Name: CHOICE, dtype: float64
Observed Market Shares: 
1    0.132026
2    0.580392
3    0.287582
Name: CHOICE, dtype: float64


The logit model seems to predicted the same share of passengers that would ride the SM trains but had incorrect market shares for the train and car. This suggests that the utility functions for the train and car are incorrect or incomplete which results in the incorrect shares for train and car 

## Task 3

Suppose that we know that market segmentation exists on income. We can then consider three
markets, namely, low income, medium income and high income that are defined as follows:
• Low Income: under CHF 50,000 (INCOME = 0 or 1)
• Medium Income: between CHF 50,000 and CHF 100,000 (INCOME = 2)
• High Income: Over CHF 100,000 (INCOME= 3).

In [121]:
# Segregate by Income
from audioop import mul


low_income = swiss_data.copy().loc[(swiss_data['INCOME'] == 0) | (swiss_data['INCOME'] == 1)]
middle_income = swiss_data.copy().loc[(swiss_data['INCOME'] == 2)]
high_income = swiss_data.copy().loc[(swiss_data['INCOME'] == 3)]

# Create a array of linspace from 0.8 to 1.2 to modify the train cost, 5 equal slices would 0.8, 0.9 ... 1.2
multiplier = np.linspace(0.8, 1.2, 5)

In [126]:
#Low income
for i in range(len(multiplier)):
    #Make new logit choice column
    low_income[f"LOGIT_CHOICE_{multiplier[i]}"] = low_income.apply(lambda row: make_logit_choice(row, multiplier[i]), axis=1)
    print(f"Logit Choice Low Income {multiplier[i]}: \n{low_income[f'LOGIT_CHOICE_{multiplier[i]}'].value_counts(normalize=True).sort_index()}")

Logit Choice Low Income 0.8: 
1    0.045926
2    0.442963
3    0.511111
Name: LOGIT_CHOICE_0.8, dtype: float64
Logit Choice Low Income 0.9: 
1    0.047407
2    0.429630
3    0.522963
Name: LOGIT_CHOICE_0.9, dtype: float64
Logit Choice Low Income 1.0: 
1    0.046420
2    0.406914
3    0.546667
Name: LOGIT_CHOICE_1.0, dtype: float64
Logit Choice Low Income 1.1: 
1    0.050864
2    0.390123
3    0.559012
Name: LOGIT_CHOICE_1.1, dtype: float64
Logit Choice Low Income 1.2: 
1    0.052346
2    0.364938
3    0.582716
Name: LOGIT_CHOICE_1.2, dtype: float64


In [123]:
#Middle income
for i in range(len(multiplier)):
    #Make new logit choice column
    middle_income[f"LOGIT_CHOICE_{multiplier[i]}"] = middle_income.apply(lambda row: make_logit_choice(row, multiplier[i]), axis=1)
    print(f"Logit Choice Middle Income {multiplier[i]}: \n{middle_income[f'LOGIT_CHOICE_{multiplier[i]}'].value_counts(normalize=True).sort_index()}")

Logit Choice Middle Income 0.8: 
1    0.022758
2    0.657028
3    0.320214
Name: LOGIT_CHOICE_0.8, dtype: float64
Logit Choice Middle Income 0.9: 
1    0.020884
2    0.657028
3    0.322088
Name: LOGIT_CHOICE_0.9, dtype: float64
Logit Choice Middle Income 1.0: 
1    0.018474
2    0.654083
3    0.327443
Name: LOGIT_CHOICE_1.0, dtype: float64
Logit Choice Middle Income 1.1: 
1    0.016064
2    0.659438
3    0.324498
Name: LOGIT_CHOICE_1.1, dtype: float64
Logit Choice Middle Income 1.2: 
1    0.013119
2    0.659438
3    0.327443
Name: LOGIT_CHOICE_1.2, dtype: float64


In [124]:
#High income
for i in range(len(multiplier)):
    #Make new logit choice column
    high_income[f"LOGIT_CHOICE_{multiplier[i]}"] = high_income.apply(lambda row: make_logit_choice(row, multiplier[i]), axis=1)
    print(f"Logit Choice High Income {multiplier[i]}: \n{high_income[f'LOGIT_CHOICE_{multiplier[i]}'].value_counts(normalize=True).sort_index()}")

Logit Choice High Income 0.8: 
1    0.018312
2    0.647859
3    0.333828
Name: LOGIT_CHOICE_0.8, dtype: float64
Logit Choice High Income 0.9: 
1    0.015095
2    0.649592
3    0.335313
Name: LOGIT_CHOICE_0.9, dtype: float64
Logit Choice High Income 1.0: 
1    0.012126
2    0.653056
3    0.334818
Name: LOGIT_CHOICE_1.0, dtype: float64
Logit Choice High Income 1.1: 
1    0.008661
2    0.658748
3    0.332591
Name: LOGIT_CHOICE_1.1, dtype: float64
Logit Choice High Income 1.2: 
1    0.007424
2    0.658500
3    0.334076
Name: LOGIT_CHOICE_1.2, dtype: float64


In general, changing the cost of SM metro will not affect market share for the middle and high income classes. It will however affect the market share of the low income classes changing the market share from 42% at 0.8 times the cost to 36% at 1.2 times the cost. Therefore, increasing train costs could have the effect of pricing out low income families from using it.